In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('../src'))
if module_path not in sys.path:
    sys.path.append(module_path)
#from vso import util, data
import astropy.units as u
from SetupSession import use_dark_theme


# Estimate star shape uniformity across field of view

In [ ]:
import vso.util
import vso.data

OBJ_NAME='SZ Lyn'
SESSION_TAG='2025/20250126'
IMAGE_ROOT = '/srv/public/img'
WORK_ROOT = '/srv/public'

session = vso.util.Session(tag=SESSION_TAG, name=OBJ_NAME)
layout = vso.util.WorkLayout(WORK_ROOT)
session_layout = layout.get_session(session)
IMAGE_DIR = vso.util.ImageLayout(IMAGE_ROOT).get_images(session).lights_dir / 'V'
sd = vso.data.StarData(layout.charts_dir)

Select image, plate solve and calibrate

In [ ]:
import ccdproc as ccdp
import vso.reduce

ifc = ccdp.ImageFileCollection(IMAGE_DIR)
path = IMAGE_DIR / ifc.summary['file'][0]
image  = vso.reduce.load_and_solve(path, session_layout.solved_dir)
matcher = vso.reduce.CalibrationMatcher(layout.calibr_dir, temp_tolerance=2*u.K)
cal = matcher.match(image.header)
reduced = vso.reduce.calibrate_image(image,
                        dark=cal.dark,
                        flat=cal.flat)


Detect point sources

In [ ]:
from astropy.stats import sigma_clipped_stats
from photutils.detection import DAOStarFinder
from astropy.coordinates import SkyCoord

mean, median, std = sigma_clipped_stats(reduced.data, sigma=3.0)
daofind = DAOStarFinder(fwhm=10.0, threshold=5.*std)

sources = daofind(reduced.data)
#sources


Run aperture photometry for all sources

In [ ]:
from photutils.aperture import CircularAperture, ApertureStats

ap = CircularAperture(zip(sources['xcentroid'], sources['ycentroid']), r=5)
ph = ApertureStats(reduced, ap)

Plot statistics acrdos FOV

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(12.80, 12.80), layout="constrained")
gs = fig.add_gridspec(2, 2)

ax = fig.add_subplot(gs[0, 0])
c = ax.tricontourf(sources['xcentroid'], sources['ycentroid'], sources['sharpness'])
plt.colorbar(c, ax=ax)
ax.set_title('DAOFIND sharpness')

ax = fig.add_subplot(gs[0, 1])
c = ax.tricontourf(sources['xcentroid'], sources['ycentroid'], sources['roundness1'])
plt.colorbar(c, ax=ax)
ax.set_title('DAOFIND roundness1')

ax = fig.add_subplot(gs[1, 0])
c = ax.tricontourf(ph.xcentroid, ph.ycentroid, ph.ellipticity.value)
plt.colorbar(c, ax=ax)
ax.set_title('ApertureStats ellipticity')

ax = fig.add_subplot(gs[1, 1])
c = ax.tricontourf(ph.xcentroid, ph.ycentroid, ph.fwhm.value)
plt.colorbar(c, ax=ax)
ax.set_title('ApertureStats FWHM')

plt.show()